<a href="https://colab.research.google.com/github/Suwandi-amin-sangaji/VGG16_Classifikasi_Sampah/blob/main/sampah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import keras, tensorflow dan VGG16
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow import keras
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model
from keras.preprocessing import image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
base_dir = '/content/drive/MyDrive/dataset/datasets-plastik'

In [ ]:
import os
# Mengambil data traning
train_dir = os.path.join(base_dir, 'train')
# mengambil data validation
val_dir = os.path.join(base_dir, 'val')

# nama p[ada setiap class/kategori
class_name = ['hdpe','ldpe','other','pet','pp','ps','pvc']

In [ ]:
# mengambil data setip classnya
train_hdpe_dir = os.path.join(base_dir, 'train/hdpe')
train_ldpe_dir = os.path.join(base_dir, 'train/ldpe')
train_other_dir = os.path.join(base_dir, 'train/other')
train_pet_dir = os.path.join(base_dir, 'train/pet')
train_pp_dir = os.path.join(base_dir, 'train/pp')
train_ps_dir = os.path.join(base_dir, 'train/ps')
train_pvc_dir = os.path.join(base_dir, 'train/pvc')

val_hdpe_dir = os.path.join(base_dir, 'val/hdpe')
val_ldpe_dir = os.path.join(base_dir, 'val/ldpe')
val_other_dir = os.path.join(base_dir, 'val/other')
val_pet_dir = os.path.join(base_dir, 'val/pet')
val_pp_dir = os.path.join(base_dir, 'val/pp')
val_ps_dir = os.path.join(base_dir, 'val/ps')
val_pvc_dir = os.path.join(base_dir, 'val/pvc')

In [ ]:
# melihat isi
train_hdpe_fnames = os.listdir(train_hdpe_dir)
train_ldpe_fnames= os.listdir(train_ldpe_dir)
train_other_fnames = os.listdir(train_other_dir)
train_pet_fnames = os.listdir(train_pet_dir)
train_pp_fnames = os.listdir(train_pp_dir)
train_ps_fnames = os.listdir(train_ps_dir)
train_pvc_fnames = os.listdir(train_pvc_dir)

print(train_hdpe_fnames[:7])
print(train_ldpe_fnames[:7])
print(train_other_fnames[:7])
print(train_pet_fnames[:7])
print(train_pp_fnames[:7])
print(train_ps_fnames[:7])
print(train_pvc_fnames[:7])


In [ ]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from PIL import Image
plt.figure(figsize=(30,14))

for i in range(7):
    directory = os.path.join(train_dir, class_name[i])
    for j in range(5):
        path = os.path.join(directory, os.listdir(directory)[j])
        img = Image.open(path)
        
        plt.subplot(7, 5, i*5 + j + 1)
        plt.imshow(img)
        
        if j == 0:
            plt.ylabel(class_name[i], fontsize=20)
        
plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[]);
plt.tight_layout()
plt.show()

In [ ]:
print("total traning hdpe images : ", len(train_hdpe_fnames))
print("total traning ldpe images : ", len(train_ldpe_fnames))
print("total traning other images : ", len(train_other_fnames))
print("total traning pet images : ", len(train_pet_fnames))
print("total traning pp images : ", len(train_pp_fnames))
print("total traning ps images : ", len(train_ps_fnames))
print("total traning pvc images : ", len(train_pvc_fnames))
print('=====================================================')

print("total validation hdpe images : ", len(val_hdpe_dir))
print("total validation ldpe images : ", len(val_ldpe_dir))
print("total validation other images : ", len(val_other_dir))
print("total validation pet images : ", len(val_pet_dir))
print("total validation pp images : ", len(val_pp_dir))
print("total validation ps images : ", len(val_ps_dir))
print("total validation pvc images : ", len(val_pvc_dir))

In [ ]:
# membuat arsitektur jaringan untuk klasifikasi
# menggunakan VGG16
conv_base = VGG16(include_top=False, weights='imagenet',  input_shape=(150,150,3))
conv_base.trainable = False

In [ ]:
# Model Jaringan dengan model Squendtial
model = tf.keras.models.Sequential([
                                    conv_base,
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dropout(0.5),
                                    tf.keras.layers.Dense(512, activation='relu'),
                                    tf.keras.layers.Dense(7, activation='softmax')
])

model.summary()

In [ ]:
plot_model(model)

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss = 'categorical_crossentropy', optimizer=opt, metrics=['acc'])

In [ ]:
# augmentasi data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# terus memasukkan dataset ke dalam model
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    class_mode='categorical',
    batch_size = 32
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150,150),
    class_mode='categorical',
    batch_size=32
)

In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    verbose=1
)

cb = [early_stopping]

In [ ]:
from keras import callbacks
history = model.fit(
    train_generator,
    epochs=50,
    validation_data = val_generator,
    callbacks= cb
)

In [ ]:
#evulate model
model.evaluate(val_generator)

In [ ]:
#plotting training values
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

train_acc = history.history['acc']
val_acc = history.history['val_acc']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(train_acc) + 1)
plt.figure(figsize=(10,6))
plt.plot(epochs, train_acc, 'b*-', label = 'Training accuracy')
plt.plot(epochs, val_acc, 'r', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(epochs, train_loss, 'b*-', label = 'Training loss')
plt.plot(epochs, val_loss, 'r', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
# SaveModel
model.save('model.h5')
model.save('/content/drive/MyDrive/dataset/datasets-plastik')

In [ ]:
classes = ['ldpe','hdpe','other','pet','pp','ps','pvc']

In [ ]:
#test prediksi
image_path = "/content/drive/MyDrive/dataset/datasets-plastik/train/other/IMG_20191229_142215.jpg"
new_img = image.load_img(image_path, target_size=(150, 150))
img = image.img_to_array(new_img)
img = np.expand_dims(img, axis=0)
prediction = model.predict(img, batch_size=10)
prediction = np.argmax(prediction,axis=1)
print(prediction)
print(classes[prediction[0]])
plt.imshow(new_img)